In [ ]:
import pandas as pd

url = "https://www.basketball-reference.com/playoffs/NBA_1950_games.html"
tables = pd.read_html(url)

# The schedule table is usually the first one
df = tables[0]
df.head()

,Date,Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 5,Unnamed: 6,Attend.,LOG,Arena,Notes
0,"Tue, Mar 21, 1950",Sheboygan Red Skins,85,Indianapolis Olympians,86,Box Score,NaN,NaN,NaN,Hinkle Fieldhouse,NaN
1,"Tue, Mar 21, 1950",New York Knicks,90,Washington Capitols,87,Box Score,NaN,NaN,NaN,Uline Arena,NaN
2,"Tue, Mar 21, 1950",Tri-Cities Blackhawks,77,Anderson Packers,89,Box Score,NaN,NaN,NaN,Anderson High School Wigwam,NaN
3,"Wed, Mar 22, 1950",Chicago Stags,75,Minneapolis Lakers,85,Box Score,NaN,NaN,NaN,St. Paul Auditorium,NaN
4,"Wed, Mar 22, 1950",Washington Capitols,83,New York Knicks,103,Box Score,NaN,NaN,NaN,Madison Square Garden (III),NaN


In [ ]:
"""
Test pipeline for 1 year of NBA playoff data (1968).
"""

import pandas as pd
from collections import defaultdict

# Load and clean the data
url = "https://www.basketball-reference.com/playoffs/NBA_1968_games.html"
df = pd.read_html(url)[0]

# Flatten headers if needed
df.columns = df.columns.droplevel(0) if isinstance(df.columns, pd.MultiIndex) else df.columns
df = df.dropna(how='all')

# Basic cleanup
df = df.rename(columns={
    "Visitor/Neutral": "Visitor",
    "PTS": "Visitor PTS",
    "Home/Neutral": "Home",
    "PTS.1": "Home PTS"
})
df['Date'] = pd.to_datetime(df['Date'] + ' 1950')  # Use fixed year since dates omit year

# Step 1: Identify games
games = []
for _, row in df.iterrows():
    team1, score1 = row['Visitor'], row['Visitor PTS']
    team2, score2 = row['Home'], row['Home PTS']
    if pd.isna(score1) or pd.isna(score2):
        continue
    score1, score2 = int(score1), int(score2)
    winner = team1 if score1 > score2 else team2
    loser = team2 if score1 > score2 else team1
    games.append({
        "date": row["Date"],
        "team1": team1,
        "score1": score1,
        "team2": team2,
        "score2": score2,
        "winner": winner,
        "loser": loser
    })

# Step 2: Group into series using team pairs
series_dict = defaultdict(list)
for game in games:
    teams = tuple(sorted([game["team1"], game["team2"]]))  # canonical team pair
    series_dict[teams].append(game)

# Step 3: Construct series summary
series_rows = []
for (team_a, team_b), game_list in series_dict.items():
    game_list = sorted(game_list, key=lambda x: x["date"])
    win_count = {team_a: 0, team_b: 0}
    scores_w = []
    scores_l = []

    for game in game_list:
        win_count[game["winner"]] += 1

        # Ensure all scores are stored as integers
        if game["winner"] == game["team1"]:
            scores_w.append(int(game["score1"]))
            scores_l.append(int(game["score2"]))
        else:
            scores_w.append(int(game["score2"]))
            scores_l.append(int(game["score1"]))

    winner = team_a if win_count[team_a] > win_count[team_b] else team_b
    loser = team_b if winner == team_a else team_a

    row = {
        "Year": 1950,
        "Series Type": "Unknown",
        "Winner Team": winner,
        "Winner Games": win_count[winner],
        "Loser Team": loser,
        "Loser Games": win_count[loser],
        "Total games": len(game_list),
    }

    # Add game scores (padded with None if < 7 games)
    for i in range(7):
        row[f"G{i+1} Score W"] = scores_w[i] if i < len(scores_w) else None
        row[f"G{i+1} Score L"] = scores_l[i] if i < len(scores_l) else None

    series_rows.append(row)

# Step 4: Final DataFrame
series_df = pd.DataFrame(series_rows)

# Convert all score columns to nullable integer (Int64)
score_cols = [col for col in series_df.columns if "Score" in col]
series_df[score_cols] = series_df[score_cols].astype("Int64")


In [ ]:
"""Display the final DataFrame for prev cell"""

series_df

,Year,Series Type,Winner Team,Winner Games,Loser Team,Loser Games,Total games,G1 Score W,G1 Score L,G2 Score W,...,G3 Score W,G3 Score L,G4 Score W,G4 Score L,G5 Score W,G5 Score L,G6 Score W,G6 Score L,G7 Score W,G7 Score L
0,1950,Unknown,San Francisco Warriors,4,St. Louis Hawks,2,6,111,106,111,...,124,109,108,107,129,103,111,106,<NA>,<NA>
1,1950,Unknown,Philadelphia 76ers,4,New York Knicks,2,6,118,110,128,...,138,132,107,98,123,105,113,97,<NA>,<NA>
2,1950,Unknown,Boston Celtics,4,Detroit Pistons,2,6,123,116,126,...,109,98,135,110,110,96,111,103,<NA>,<NA>
3,1950,Unknown,Los Angeles Lakers,4,Chicago Bulls,1,5,109,101,111,...,104,98,93,87,122,99,<NA>,<NA>,<NA>,<NA>
4,1950,Unknown,Los Angeles Lakers,4,San Francisco Warriors,0,4,133,105,115,...,128,124,106,100,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,1950,Unknown,Boston Celtics,4,Philadelphia 76ers,3,7,127,118,115,...,122,114,110,105,122,104,114,106,100,96
6,1950,Unknown,Boston Celtics,4,Los Angeles Lakers,2,6,107,101,123,...,127,119,118,105,120,117,124,109,<NA>,<NA>


In [1]:
"""Iterate pipeline over years_list and save to Excel"""


import pandas as pd
from collections import defaultdict
import time

years_list = range(1968, 1977)

all_series_rows = []

for year in years_list:
    print(f"Processing year: {year}")
    
    # Change to ABA or ABA depending on the required data
    url = f"https://www.basketball-reference.com/playoffs/ABA_{year}_games.html"
    
    try:
        df = pd.read_html(url)[0]
    except Exception as e:
        print(f"Failed to load year {year}: {e}")
        continue

    df.columns = df.columns.droplevel(0) if isinstance(df.columns, pd.MultiIndex) else df.columns
    df = df.dropna(how='all')

    if "Visitor/Neutral" not in df.columns or "Home/Neutral" not in df.columns:
        print(f"Skipping year {year}: Missing expected columns")
        continue

    df = df.rename(columns={
        "Visitor/Neutral": "Visitor",
        "PTS": "Visitor PTS",
        "Home/Neutral": "Home",
        "PTS.1": "Home PTS"
    })

    # Some years already have full date
    if not df["Date"].astype(str).str.contains(str(year)).any():
        df['Date'] = pd.to_datetime(df['Date'] + f" {year}", errors='coerce')
    else:
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    # Step 1: Identify games
    games = []
    for _, row in df.iterrows():
        team1, score1 = row['Visitor'], row['Visitor PTS']
        team2, score2 = row['Home'], row['Home PTS']
        if pd.isna(score1) or pd.isna(score2):
            continue
        try:
            score1, score2 = int(score1), int(score2)
        except:
            continue
        winner = team1 if score1 > score2 else team2
        loser = team2 if score1 > score2 else team1
        games.append({
            "date": row["Date"],
            "team1": team1,
            "score1": score1,
            "team2": team2,
            "score2": score2,
            "winner": winner,
            "loser": loser
        })

    # Step 2: Group into series using team pairs
    series_dict = defaultdict(list)
    for game in games:
        teams = tuple(sorted([game["team1"], game["team2"]]))
        series_dict[teams].append(game)

    # Step 3: Construct series summary
    for (team_a, team_b), game_list in series_dict.items():
        game_list = sorted(game_list, key=lambda x: x["date"])
        win_count = {team_a: 0, team_b: 0}

        for game in game_list:
            win_count[game["winner"]] += 1

        # Determine series winner and loser
        winner = team_a if win_count[team_a] > win_count[team_b] else team_b
        loser = team_b if winner == team_a else team_a

        scores_w = []
        scores_l = []

        # Collect scores from series winner/loser perspective
        for game in game_list:
            if game["team1"] == winner:
                scores_w.append(int(game["score1"]))
                scores_l.append(int(game["score2"]))
            elif game["team2"] == winner:
                scores_w.append(int(game["score2"]))
                scores_l.append(int(game["score1"]))
            else:
                raise ValueError("Unexpected team names when collecting scores")

        row = {
            "Year": year,
            "Series Type": "Unknown",  # Placeholder
            "Winner Team": winner,
            "Winner Games": win_count[winner],
            "Loser Team": loser,
            "Loser Games": win_count[loser],
            "Total games": len(game_list),
        }

        # Add 7 score columns each for winner and loser
        for i in range(7):
            row[f"G{i+1} Score W"] = scores_w[i] if i < len(scores_w) else None
        for i in range(7):
            row[f"G{i+1} Score L"] = scores_l[i] if i < len(scores_l) else None

        all_series_rows.append(row)

    time.sleep(1)  # Be polite to the server

# Combine and clean final DataFrame
all_series_df = pd.DataFrame(all_series_rows)
score_cols = [col for col in all_series_df.columns if "Score" in col]
all_series_df[score_cols] = all_series_df[score_cols].astype("Int64")

# Save to Excel
output_file = "aba_playoff_series_1968_1976.xlsx"
all_series_df.to_excel(output_file, index=False)
print(f"Saved to {output_file}")


Processing year: 1968
Processing year: 1969
Processing year: 1970
Processing year: 1971
Processing year: 1972
Processing year: 1973
Processing year: 1974
Processing year: 1975
Processing year: 1976
Saved to aba_playoff_series_1968_1976.xlsx


In [2]:
1/7.

0.14285714285714285